Bài 2. MLP – Phân loại chữ số MNIST
• Bộ dữ liệu: `torchvision.datasets.MNIST` (60k train/10k test).
• Mục tiêu: Xây MLP 2–3 tầng ẩn cho phân loại 10 lớp.
• Nhiệm vụ:
• Chuẩn hoá ảnh về [0,1]; flatten 28×28.
• Kiến trúc gợi ý: 784→512→256→10; ReLU + Dropout.
• Huấn luyện với Adam, lr=1e-3; Early stopping.
• Báo cáo accuracy & biểu đồ loss/accuracy theo epoch.
• Đánh giá: Top-1 Accuracy trên test; tốc độ hội tụ.
• Ràng buộc: Epoch≤20; batch=128.
• Sản phẩm nộp: Mã nguồn, checkpoint, biểu đồ; 1 trang phân tích overfitting/regularization.
• Nâng cao (tuỳ chọn):
• Thử GELU/BatchNorm.
• Tối ưu số tầng & units.


# Cell 1 — Imports & cấu hình


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

import numpy as np
import matplotlib.pyplot as plt
import os

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

BATCH_SIZE = 128
LR = 1e-3
MAX_EPOCHS = 20
PATIENCE = 3  # early stopping


ModuleNotFoundError: No module named 'torch'

In [ ]:
# Cell 2 — Dataset & DataLoader
transform = transforms.Compose([
    transforms.ToTensor(),  # [0,1]
    transforms.Lambda(lambda x: x.view(-1))  # flatten 28x28 -> 784
])

train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

# Tách train/val (50k/10k)
train_size = 50000
val_size = len(train_dataset) - train_size
train_data, val_data = random_split(train_dataset, [train_size, val_size], generator=torch.Generator().manual_seed(SEED))

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Train: {len(train_data)}, Val: {len(val_data)}, Test: {len(test_dataset)}")


In [ ]:
# Cell 3 — MLP Model
class MLP(nn.Module):
    def __init__(self, input_dim=784, hidden1=512, hidden2=256, num_classes=10, dropout=0.5):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden1),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden1, hidden2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden2, num_classes)
        )
    def forward(self, x):
        return self.net(x)

model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)


In [ ]:
# Cell 4 — Training loop with Early Stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, max_epochs=20, patience=3):
    best_val_loss = float("inf")
    patience_counter = 0
    history = {"train_loss": [], "val_loss": [], "train_acc": [], "val_acc": []}
    
    for epoch in range(1, max_epochs+1):
        # Train
        model.train()
        train_loss, correct, total = 0, 0, 0
        for X, y in train_loader:
            optimizer.zero_grad()
            outputs = model(X)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * X.size(0)
            _, preds = outputs.max(1)
            correct += preds.eq(y).sum().item()
            total += y.size(0)
        train_loss /= total
        train_acc = correct / total
        
        # Validation
        model.eval()
        val_loss, correct, total = 0, 0, 0
        with torch.no_grad():
            for X, y in val_loader:
                outputs = model(X)
                loss = criterion(outputs, y)
                val_loss += loss.item() * X.size(0)
                _, preds = outputs.max(1)
                correct += preds.eq(y).sum().item()
                total += y.size(0)
        val_loss /= total
        val_acc = correct / total
        
        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        history["train_acc"].append(train_acc)
        history["val_acc"].append(val_acc)
        
        print(f"Epoch {epoch:02d}: train_loss={train_loss:.4f}, val_loss={val_loss:.4f}, train_acc={train_acc:.4f}, val_acc={val_acc:.4f}")
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), "mlp_mnist_checkpoint.pt")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered.")
                break
    return history

history = train_model(model, train_loader, val_loader, criterion, optimizer, MAX_EPOCHS, PATIENCE)


In [ ]:
# Cell 5 — Load best checkpoint & Evaluate on test
best_model = MLP()
best_model.load_state_dict(torch.load("mlp_mnist_checkpoint.pt"))
best_model.eval()

correct, total = 0, 0
with torch.no_grad():
    for X, y in test_loader:
        outputs = best_model(X)
        _, preds = outputs.max(1)
        correct += preds.eq(y).sum().item()
        total += y.size(0)

test_acc = correct / total
print(f"Test Accuracy: {test_acc:.4f}")


Quan sát loss/accuracy: 

    Trong quá trình huấn luyện, train loss giảm nhanh, train accuracy tăng mạnh. Tuy nhiên, val loss có thể ngừng giảm sớm hơn, và val accuracy có thể dừng lại hoặc giảm nhẹ sau vài epoch. Đây là dấu hiệu overfitting: mô hình học quá kỹ dữ liệu train, giảm khả năng tổng quát hóa.

Regularization áp dụng:

    Dropout (0.5): giúp giảm overfitting bằng cách ngẫu nhiên bỏ neuron trong quá trình huấn luyện, buộc mạng học các biểu diễn đa dạng.

Early stopping: 

    dừng huấn luyện khi val loss không cải thiện, tránh mô hình tiếp tục overfit.

Adam optimizer: 

    hội tụ nhanh, nhưng dễ overfit nếu không có dropout/early stopping.

Kết quả: 

    Với kiến trúc 784→512→256→10, dropout 0.5, lr=1e-3, batch=128, mô hình thường đạt ~97–98% test accuracy trong ≤20 epoch. Đây là mức tốt cho MLP trên MNIST (CNN có thể đạt >99%).

Nâng cao:


BatchNorm: giúp ổn định phân phối kích hoạt, tăng tốc hội tụ.

GELU: có thể cải thiện nhẹ so với ReLU.

Tối ưu số tầng/units: giảm số units có thể giảm overfitting, tăng tốc độ.

Kết luận: 

        MLP với dropout + early stopping đã kiểm soát overfitting khá tốt. Nếu bỏ dropout, train acc có thể đạt gần 100% nhưng val/test acc giảm. Điều này minh họa vai trò quan trọng của regularization trong mạng nơ-ron.